In [2]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [4]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                    padding="SAME")

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                    padding='SAME')

In [21]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

In [22]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [23]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                            logits=model, labels=Y))
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [24]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [25]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [ ]:
for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1) # -1 unknown. https://stackoverflow.com/questions/34642595/tensorflow-strides-argument
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict = {X: batch_xs,
                                            Y: batch_ys,
                                            keep_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch + 1),
          'Ave. cost = ', '{:.3f}'.format(total_cost / total_batch))
print('Learning finisied')

In [ ]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('accuracy', sess.run(accuracy,
                           feed_dict = {X: mnist.test.images.reshape(
                           )}))